## import modules
## Load the datasets which are scraped
## Lemmatizer stopword removal modules 
## clean skills dataset 
## filter the data for validation
## clean jobs dataset 
## phrase matcher model to extract skills from job description 
## create job titles using job progression dictionary and declare set of properly defined titles  
## clean job titles by fuzzy matching and normalizing against job preogression titles
## clean course data 
## Lemmitization and cleaning columns
## merge all description into one
## save processed datasets

In [42]:
import pandas as pd
pd.set_option('display.max_columns', None) 
from os import listdir
import os
import glob
import spacy
from collections import Counter
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from spacy.scorer import Scorer
import re
import nlp

[nltk_data] Downloading package stopwords to /Users/nyzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
## Load the datasets which are scraped

#load data
job_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/raw/job_data"
courses_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/raw/courses_data"
skills_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/skills_dataset/skills_df_updated.csv"

## courses
files_c= [f for f in listdir(courses_path) if f.endswith(".csv")]
df_c = pd.concat([pd.read_csv(os.path.join(courses_path,f)) for f in files_c])

## skills 
#files_s= [f for f in listdir(skills_path) if f.endswith(".csv")]
df_s = pd.read_csv(skills_path)

## jobs
df_j_main_list=[]
job_dir = listdir(job_path)
job_dir.remove('.DS_Store')
for job in job_dir:
    files_j= [f for f in listdir(os.path.join(job_path,job)) if f.endswith(".csv")]
    df_j = pd.concat([pd.read_csv(os.path.join(job_path,job,f)) for f in files_j])
    df_j["searched_title"]=job
    df_j_main_list.append(df_j)
df_j = pd.concat(df_j_main_list)

In [53]:
df_j

,Unnamed: 0,company,viewJobLink,companyBrandingAttributes,companyOverviewLink,companyRating,companyReviewCount,displayTitle,employerAssistEnabled,employerResponsive,expired,featuredCompanyAttributes,featuredEmployer,featuredEmployerCandidate,feedId,formattedLocation,formattedRelativeTime,highVolumeHiringModel,hiringEventJob,indeedApplyEnabled,indeedApplyable,isJobVisited,isMobileThirdPartyApplyable,isNoResumeJob,isSubsidiaryJob,jobCardRequirementsModel,jobLocationCity,jobLocationState,jobTypes,locationCount,newJob,normTitle,openInterviewsInterviewsOnTheSpot,openInterviewsJob,openInterviewsOffersOnTheSpot,openInterviewsPhoneJob,overrideIndeedApplyText,pubDate,rankingScoresModel,remoteLocation,resumeMatch,salarySnippet,screenerQuestionsURL,showAttainabilityBadge,showCommutePromo,showEarlyApply,showJobType,showRelativeDate,showSponsoredLabel,showStrongerAppliedLabel,smartFillEnabled,smbD2iEnabled,snippet,sponsored,taxoAttributes,taxoAttributesDisplayLimit,taxoLogAttributes,taxonomyAttributes,thirdPartyApplyUrl,title,translatedAttributes,translatedCmiJobTags,truncatedCompany,urgentlyHiring,vjFeaturedEmployerCandidate,jobDescription,extractedSalary,searched_title
0,0,Community Hospital,/viewjob?jk=4d8992b619aa6b59&from=vjs&tk=1hf0d...,{'headerImageUrl': 'https://d2q79iu7y748jz.clo...,https://www.indeed.com/cmp/Community-Hospital-...,3.1,39.0,System Chief Quality Officer/Regulatory Compli...,False,False,False,{},False,False,50461,"Oklahoma City, OK 73159",30+ days ago,{'highVolumeHiring': False},False,True,True,False,False,False,False,"{'additionalRequirementsCount': 0, 'requiremen...",Oklahoma City,OK,[],1,False,Chief Quality Officer,False,False,False,False,True,1674540000000,"{'bid': 516053, 'eApply': 0.0009227544, 'eQual...",False,False,"{'currency': 'USD', 'salaryTextFormatted': False}",iq://10c7fa972cfad66fc91d?v=1,False,False,False,False,True,False,False,False,False,This position is responsible for facilitating ...,True,[],3,[],"[{'attributes': [], 'label': 'dynamic-attribut...",http://www.indeed.com//applystart?jk=4d8992b61...,System Chief Quality Officer/Regulatory Compli...,[],[],Community Hospital,False,False,Updated pay scales as of August 2022.\nWe’re l...,NaN,Chief Intelligence Officer
1,1,Mesalands Community College,/viewjob?jk=808042413e184228&from=vjs&tk=1hf0d...,{},https://www.indeed.com/cmp/Mesalands-Community...,3.8,14.0,Chief Financial Officer,True,False,False,{},False,False,50461,"Tucumcari, NM 88401",30+ days ago,{'highVolumeHiring': False},False,True,True,False,False,False,False,"{'additionalRequirementsCount': -2, 'requireme...",Tucumcari,NM,[],1,False,Chief Financial Officer,False,False,False,False,True,1695618000000,"{'bid': 364162, 'eApply': 0.00069609965, 'eQua...",False,False,"{'currency': 'USD', 'salaryTextFormatted': Fal...",iq://d37113be08ab5ba23aef?v=1,False,False,False,False,True,False,False,False,False,FULL-TIME/ PART-TIME: Full Time.\n Mesalands C...,True,[],3,[],"[{'attributes': [], 'label': 'dynamic-attribut...",http://www.indeed.com//applystart?jk=808042413...,Chief Financial Officer,[],[],Mesalands Community College,True,False,MESALANDS COMMUNITY COLLEGE\nPosition Announce...,"{'max': 90000, 'min': 65000, 'type': 'yearly'}",Chief Intelligence Officer
2,2,Cornerstone Healthcare Group,/viewjob?jk=662ea1d8fa14338a&from=vjs&tk=1hf0d...,{'headerImageUrl': 'https://d2q79iu7y748jz.clo...,https://www.indeed.com/cmp/Kindred-Hospitals-3...,3.3,5192.0,Chief Operating Officer / Chief Clinical Officer,True,False,False,{'logoUrl': 'https://d2q79iu7y748jz.cloudfront...,True,False,50461,"Indianapolis, IN 46222",5 days ago,{'highVolumeHiring': False},False,True,True,False,False,False,False,"{'additionalRequirementsCount': 0, 'requiremen...",Indianapolis,IN,[],5,True,Chief Operating Officer,False,False,False,False,True,1699250400000,"{'bid': 76882, 'eApply': 0.0010758947, 'eQuali...",False,False,"{'currency': 'USD', 'salaryTextFormatted': False}",iq://655a2395ef8200b4b70f?v=1,False,False,False,False,True,False,

In [54]:
## Lemmatizer stopword removal modules 




# Ensure you have the necessary NLTK tokens downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Load the spaCy model
nlp = spacy.load('en_core_web_md')

def preprocess_text_spacy(text):
    # Use spaCy's pipeline for processing the passed text
    doc = nlp(text)
    preprocessed_text= []
    # Return a preprocessed version of the text (for instance, lemmatized, lowercased, no stop words/punctuations)
    for token in doc:
        # Skip stop words and punctuation
        if token.is_stop or token.is_punct or token.is_space:
            continue
        # Special case for 'data' to keep it in plural form
        if token.lemma_ == 'datum':
            preprocessed_text.append('data')
        else:
            preprocessed_text.append(token.text.lower())

    return ' '.join(preprocessed_text)


# Function to clean and lemmatize job title text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = nltk.word_tokenize(text)
    #words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

def clean_lem_stop(df,column_name):
    df[column_name] = df[column_name].apply(preprocess_text_spacy)
    df[column_name] = df[column_name].apply(clean_text)
    return df





[nltk_data] Downloading package punkt to /Users/nyzy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nyzy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/nyzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
## clean skills dataset 

#clean skills dataset 

skills = pd.DataFrame(df_s[df_s["Unnamed: 5"].notna()]["Unnamed: 5"].unique())
skills.columns=["skills"]
#skills["skills"] = skills["skills"].apply(preprocess_text_spacy)
skills["skills"] = skills["skills"].apply(clean_text)
skills_list = skills["skills"].tolist()
skills_list.append('python')
skills_list.append('python programming')
skills_list.append('statistical')
skills_list.append("r programming")

for ach in ["docker",
 "neural network","matlab","google bard ai","ai governance","machine learning","tensorflow","computer vision","prompts","generate prompts","generative","generative ai","nlp","natural language processing","langchain",
 "pytorch","llm","scala","opencv"]:
    skills_list.append(ach)



In [39]:
df_s_skills_list = pd.DataFrame(skills_list, columns=['skills_list'])

df_s_skills_list.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/skills_dataset/skills_list_df.csv")



In [28]:
## filter the data for validation
df_j = df_j[:100]
df_c = df_c[:100]

In [55]:
## clean jobs dataset 

def clean_job_df(df):
    df = clean_lem_stop(df,"title")
    df = clean_lem_stop(df,"jobDescription")
    job_descriptions = df_j["jobDescription"].tolist()
    return df,job_descriptions

df_j_cleaned,job_descriptions = clean_job_df(df_j)

columns_of_interest = ['company',
 'companyRating',
 'companyReviewCount',
 'displayTitle',
 'employerAssistEnabled',
 'employerResponsive',
 'extractedSalary',
 'featuredEmployer',
 'featuredEmployerCandidate',
 'formattedLocation',
 'formattedRelativeTime',
 'highVolumeHiringModel',
 'hiringEventJob',
 'indeedApplyEnabled',
 'indeedApplyable',
 'isJobVisited',
 'isMobileThirdPartyApplyable',
 'isNoResumeJob',
 'isSubsidiaryJob',
 'jobCardRequirementsModel',
 'jobLocationCity',
 'jobLocationState',
 'locationCount',
 'newJob',
 'normTitle',
 'openInterviewsInterviewsOnTheSpot',
 'openInterviewsJob',
 'openInterviewsOffersOnTheSpot',
 'openInterviewsPhoneJob',
 'overrideIndeedApplyText',
 'remoteLocation',
 'resumeMatch',
 'salarySnippet',
 'showAttainabilityBadge',
 'showCommutePromo',
 'showEarlyApply',
 'showJobType',
 'showRelativeDate',
 'showSponsoredLabel',
 'showStrongerAppliedLabel',
 'smartFillEnabled',
 'smbD2iEnabled',
 'snippet',
 'sponsored',
 'title',
 'truncatedCompany',
 'urgentlyHiring',
 'vjFeaturedEmployerCandidate',
 'jobDescription','searched_title']


import ast 
import numpy as np
def salary_to_dict(row):
    if pd.isna(row):
        # Return a dictionary with NaN values if the row is NaN
        return {'max': np.nan, 'min': np.nan, 'type': np.nan}
    elif isinstance(row, dict):
        # Return the row as is if it's already a dictionary
        return row
    else:
        # If the row is a string representation of a dictionary (assumed if not NaN or dict),
        # safely evaluate it to a dictionary here
        # Note: Be cautious with `eval`. Here it's mentioned for potential string to dict conversion.
        # In a secure context, confirm the string format and consider `ast.literal_eval` instead.
        try:
            # Convert string to dictionary safely
            dict_row = eval(row)
            return dict_row if isinstance(dict_row, dict) else {'max': np.nan, 'min': np.nan, 'type': np.nan}
        except:
            # In case of error during eval, return NaN values
            return {'max': np.nan, 'min': np.nan, 'type': np.nan}

# Apply the function to each row of the 'extractedSalary' column
salary_dicts = df_j_cleaned['extractedSalary'].apply(salary_to_dict)
# Now that we have a series of dictionaries, use `json_normalize` to create a DataFrame
salary_df = pd.json_normalize(salary_dicts)

# Concatenate the new DataFrame with the original one
df_j_cleaned = pd.concat([df_j_cleaned.drop('extractedSalary', axis=1).reset_index(), salary_df], axis=1)

#df["extractedSalary"]=df["extractedSalary"].astype(str)
#df["extractedSalary"]=df['extractedSalary'].apply(ast.literal_eval)

import ast 
import numpy as np
def salary_snippet_to_dict(row):
    if pd.isna(row):
        # Return a dictionary with NaN values if the row is NaN
        return {'currency': np.nan, 'salaryTextFormatted': np.nan, 'source': np.nan,'text':np.nan}
    elif isinstance(row, dict):
        # Return the row as is if it's already a dictionary
        return row
    else:
        # If the row is a string representation of a dictionary (assumed if not NaN or dict),
        # safely evaluate it to a dictionary here
        # Note: Be cautious with `eval`. Here it's mentioned for potential string to dict conversion.
        # In a secure context, confirm the string format and consider `ast.literal_eval` instead.
        try:
            # Convert string to dictionary safely
            dict_row = eval(row)
            return dict_row if isinstance(dict_row, dict) else {'max': np.nan, 'min': np.nan, 'type': np.nan}
        except:
            # In case of error during eval, return NaN values
            return {'currency': np.nan, 'salaryTextFormatted': np.nan, 'source': np.nan,'text':np.nan}

# Apply the function to each row of the 'extractedSalary' column
salary_snippet_dicts = df_j_cleaned['salarySnippet'].apply(salary_snippet_to_dict)
# Now that we have a series of dictionaries, use `json_normalize` to create a DataFrame
salary_snippet_df = pd.json_normalize(salary_snippet_dicts)

# Concatenate the new DataFrame with the original one
df_j_cleaned = pd.concat([df_j_cleaned.drop('salarySnippet', axis=1).reset_index(), salary_snippet_df], axis=1)

#df["extractedSalary"]=df["extractedSalary"].astype(str)
#df["extractedSalary"]=df['extractedSalary'].apply(ast.literal_eval)


In [56]:
## phrase matcher model to extract skills from job description 



#phrase matcher model for skills

import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.util import filter_spans

def phrase_matcher_model(description,skills_list):
    nlp = spacy.load("en_core_web_md")  # Load the model
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")  # Create the matcher object

    # Assuming 'skills_list' is a list of skills, and 'job_descriptions' is a list containing job descriptions

    # Add patterns to the matcher. Patterns are made by converting each skill string into a Doc object
    patterns = [nlp.make_doc(skill) for skill in skills_list]
    matcher.add("Skills", patterns)

    # Process the job description to create a Spacy Doc
    doc = nlp(description)

    # Match the patterns to the doc
    matches = matcher(doc)

    # Create Span objects for the matched sequences
    spans = [Span(doc, start, end, label="SKILL") for match_id, start, end in matches]

    # Filter the spans to remove overlaps
    filtered_spans = filter_spans(spans)

    # Now you can create new entities in the doc using the filtered spans
    doc.ents = filtered_spans  # Overwrite or append to doc.ents with the non-overlapping skill entities
    entities_extracted = []
    # Print the entities in the document
    for ent in doc.ents:
        entities_extracted.append(ent.text)
    '''
    for each in matches:
        #print(each)
        if each.lower_ == "statistical":
            for skills_identify in skills_list:
                if each.lower_ in skills_identify:
                    #print(skills_identify)
                    entities_extracted.append(skills_identify)
    '''
    return set(entities_extracted)

df_j_cleaned["skills_tagged"]=df_j_cleaned["jobDescription"].apply(lambda x:phrase_matcher_model(x,skills_list))




In [57]:
## clean job titles using job progression dictionary and declare set of properly defined titles  


#job progression dictionary
# Define the original dictionary provided by the user

Job_progression_dictionary =  {
"Junior Data Analyst" : ["Data Analyst", "Senior Data Analyst", "Data Engineer", "Senior Data Engineer", "Lead Data Engineer", "Data Architect", "Data Manager", "Director of Analytics", "Chief Data Officer"],
"Junior Data Scientist" : ["Data Scientist", "Senior Data Scientist", "Lead Data Scientist", "Principal Data Scientist", "Data Science Manager", "Director of Data Science", "Vice President of Data Science", "Chief Data Scientist"],
"Junior Data Engineer" : ["Data Engineer", "Senior Data Engineer", "Data Pipeline Architect", "Data Engineering Manager", "Director of Data Engineering", "Chief Technology Officer (CTO)"],
"Business Intelligence Analyst" : ["Senior Business Intelligence Analyst", "Business Intelligence Manager", "Director of Business Intelligence", "Vice President of Business Intelligence", "Chief Intelligence Officer"],
"Machine Learning Engineer" : ["Senior Machine Learning Engineer", "Machine Learning Architect", "Machine Learning Manager", "Head of Machine Learning", "Chief AI Officer"],
"Data Analyst" : ["Data Engineer", "Senior Data Engineer", "Data Architect", "Senior Data Architect", "Enterprise Architect", "Chief Technology Officer (CTO)"],
"Data Analyst" : ["Big Data Engineer", "Senior Big Data Engineer", "Big Data Architect", "Head of Big Data", "Director of Data Engineering", "Chief Information Officer (CIO)"],
"Junior AI Developer" : ["AI Developer", "Senior AI Developer", "AI Architect", "AI Project Manager", "Head of AI", "Chief AI Officer"],
"Junior DBA" : ["Mid-level DBA", "Senior DBA", "Database Manager", "Data Architect", "Director of Database Management", "Chief Information Officer"],
"Junior Statistician" : ["Statistician" , "Senior Statistician", "Quantitative Analyst", "Senior Quantitative Analyst", "Quantitative Research Manager", "Director of Quantitative Research"],
"Junior Business Analyst" : ["Business Analyst", "Senior Business Analyst", "Business Analysis Manager", "Business Intelligence Analyst", "Director of Business Analysis", "Chief Strategy Officer"],
"Research Assistant" : ["Research Analyst", "Research Scientist", "Senior Research Scientist", "Principal Scientist", "Director of Research", "Chief Science Officer"],
"Data Analyst" : ["Product Analyst", "Data Product Manager", "Senior Data Product Manager", "Director of Product Management", "Vice President of Product", "Chief Product Officer"]
}

original_dict = {
    "Junior Data Analyst": ["Data Analyst", "Senior Data Analyst", "Data Engineer", "Senior Data Engineer", 
                            "Lead Data Engineer", "Data Architect", "Data Manager", "Director of Analytics", 
                            "Chief Data Officer"],
    "Junior Data Scientist": ["Data Scientist", "Senior Data Scientist", "Lead Data Scientist", 
                              "Principal Data Scientist", "Data Science Manager", "Director of Data Science", 
                              "Vice President of Data Science", "Chief Data Scientist"],
    "Junior Data Engineer": ["Data Engineer", "Senior Data Engineer", "Data Pipeline Architect", 
                             "Data Engineering Manager", "Director of Data Engineering", "Chief Technology Officer"],
    "Business Intelligence Analyst": ["Senior Business Intelligence Analyst", "Business Intelligence Manager", 
                                      "Director of Business Intelligence", "Vice President of Business Intelligence", 
                                      "Chief Intelligence Officer"],
    "Machine Learning Engineer": ["Senior Machine Learning Engineer", "Machine Learning Architect", 
                                  "Machine Learning Manager", "Head of Machine Learning", "Chief AI Officer"],
    "Junior AI Developer": ["AI Developer", "Senior AI Developer", "AI Architect", "AI Project Manager", 
                            "Head of AI", "Chief AI Officer"],
    "Junior DataBase Administrator": ["DataBase Administrator", "Senior DataBase Administrator", "Database Manager", "Data Architect", "Director of Database Management", 
                   "Chief Information Officer"],
    "Junior Statistician": ["Statistician", "Senior Statistician", "Quantitative Analyst", "Senior Quantitative Analyst", 
                            "Quantitative Research Manager", "Director of Quantitative Research"],
    "Junior Business Analyst": ["Business Analyst", "Senior Business Analyst", "Business Analysis Manager", 
                                "Business Intelligence Analyst", "Director of Business Analysis", "Chief Strategy Officer"],
    "Research Assistant": ["Research Analyst", "Research Scientist", "Senior Research Scientist", "Principal Scientist", 
                           "Director of Research", "Chief Science Officer"],
    # Multiple entries for Data Analyst have been combined to include all unique progressions
    "Data Analyst": ["Data Engineer", "Senior Data Engineer", "Lead Data Engineer", "Data Architect", 
                     "Senior Data Architect", "Enterprise Architect", "Product Analyst", "Data Product Manager", 
                     "Senior Data Product Manager", "Director of Product Management", "Vice President of Product", 
                     "Chief Product Officer", "Big Data Engineer", "Senior Big Data Engineer", "Big Data Architect", 
                     "Head of Big Data", "Director of Data Engineering", "Chief Information Officer"]
}

# Define lateral moves for the given roles
lateral_moves = {
    "Data Analyst": ["Business Intelligence Analyst", "Machine Learning Engineer"],
    "Data Scientist": ["Data Engineer", "AI Developer"],
    "Data Engineer": ["Machine Learning Engineer", "Big Data Engineer"],
    "Business Intelligence Analyst": ["Data Analyst", "Data Scientist"],
    "Machine Learning Engineer": ["Data Scientist", "AI Developer"],
    "AI Developer": ["Machine Learning Engineer", "Data Engineer"],
    "Data Base Administrator": ["Data Engineer", "Data Analyst"],
    "Statistician": ["Data Analyst", "Data Scientist"],
    "Business Analyst": ["Data Analyst", "Business Intelligence Analyst"],
    "Research Analyst": ["Data Scientist", "Statistician"]
}

# Since we want to include lateral moves for each value in the original dictionary, 
# we will create a function that merges the direct progressions and lateral moves into one list.

# Function to merge progression and lateral moves
def merge_progression_and_lateral_moves(direct_progression, lateral_move_titles):
    # Start with direct progression
    full_progression = direct_progression.copy()
    
    # Add lateral moves for each title in the direct progression if they exist
    for title in direct_progression:
        lateral_titles = lateral_moves.get(title, [])
        for lateral_title in lateral_titles:
            if lateral_title not in full_progression:  # Avoid duplicates
                full_progression.append(lateral_title)
    
    return full_progression

# Function to build a full job progression dictionary for each title
def build_full_progression_dict(original_dict, lateral_moves):
    full_progression_dict = {}
    
    # Iterate over each starting job title
    for start_title, progression in original_dict.items():
        # Get the full progression for the starting title
        full_progression = merge_progression_and_lateral_moves(progression, lateral_moves)
        
        # Add the full progression to the dictionary for the starting title
        full_progression_dict[start_title] = full_progression
        
        # Now iterate over each job within the progression to build their own progression paths
        for i, title in enumerate(progression):
            if title not in full_progression_dict:  # Only add if it doesn't already exist to avoid overwriting
                # Get the progression for this title (which is the rest of the list after this title)
                next_progression = merge_progression_and_lateral_moves(progression[i + 1:], lateral_moves)
                full_progression_dict[title] = next_progression
    
    return full_progression_dict

# Building the full job progression dictionary
full_job_progression_dict = build_full_progression_dict(original_dict, lateral_moves)

# Sorting the dictionary for better readability
sorted_full_job_progression_dict = {k: full_job_progression_dict[k] for k in sorted(full_job_progression_dict)}


unique_job_title_full = set()
for key,value in Job_progression_dictionary.items():
    unique_job_title_full.add(key)
    for each in value:
        unique_job_title_full.add(each)


unique_job_title = set()
for key,value in sorted_full_job_progression_dict.items():
    unique_job_title.add(key)
    for each in value:
        unique_job_title.add(each)

unique_job_title.difference(unique_job_title_full)

sorted_full_job_progression_dict

sorted_full_job_progression_dict_lower={}
for k,v in sorted_full_job_progression_dict.items():
    sorted_full_job_progression_dict_lower[k.lower()]=[each.lower() for each in v]

sorted_full_job_progression_dict_lower

{'ai architect': ['ai project manager', 'head of ai', 'chief ai officer'],
 'ai developer': ['senior ai developer',
  'ai architect',
  'ai project manager',
  'head of ai',
  'chief ai officer'],
 'ai project manager': ['head of ai', 'chief ai officer'],
 'big data architect': ['head of big data',
  'director of data engineering',
  'chief information officer'],
 'big data engineer': ['senior big data engineer',
  'big data architect',
  'head of big data',
  'director of data engineering',
  'chief information officer'],
 'business analysis manager': ['business intelligence analyst',
  'director of business analysis',
  'chief strategy officer',
  'data analyst',
  'data scientist'],
 'business analyst': ['senior business analyst',
  'business analysis manager',
  'business intelligence analyst',
  'director of business analysis',
  'chief strategy officer',
  'data analyst',
  'data scientist'],
 'business intelligence analyst': ['senior business intelligence analyst',
  'business i

In [40]:
import json
# Saving the jon progress dictionary to a JSON file
Job_progression_dictionary_file_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_progression_dictionary.json"

with open(Job_progression_dictionary_file_path, 'w') as json_file:
    json.dump(sorted_full_job_progression_dict_lower, json_file)


In [58]:
#clean job titles by fuzzy matching and normalizing against job preogression titles
import re
import numpy as np
from fuzzywuzzy import process
#base_job_titles = list(sorted_full_job_progression_dict.keys())  # Add any other base titles as necessary
#scraped_titles = df_j_cleaned["displayTitle"].unique().tolist()

# Function to normalize job titles
def normalize_title(title):
    # Lowercase and remove non-alphanumeric characters, replace with spaces
    title = re.sub(r'[^a-z0-9]', ' ', title.lower())
    # Remove extra whitespace
    title = re.sub(r'\s+', ' ', title).strip()

        # Remove common prefixes/suffixes
    #title = re.sub(r'\b(senior|junior|associate|expert|technical|lead|l\d+)\b', '', title)
    title = re.sub(r'\b(sr|senior|expert|technical|lead|l\d+)\b', 'Senior', title)
    title = re.sub(r'\b(vp|vice president|l\d+)\b', 'Senior', title)
    title = re.sub(r'\b(III|6|5|4|3|l\d+)\b', 'Senior', title)
    # Replace specific terms with standardized equivalents
    title = re.sub(r'\b(sr\.?|senior)\b', 'senior', title)  # Replace 'sr' or 'senior' with 'senior'
    title = re.sub(r'\b(jr\.?|junior)\b', 'junior', title) 

    # Convert to lower case and remove special characters
    title = re.sub(r'[^a-z\s]', '', title.lower())

    # Strip extra whitespace
    title = re.sub(r'\s+', ' ', title).strip()
    return title

# Create a set of base job titles from your job progression dictionary
base_job_titles = set()
for titles_list in sorted_full_job_progression_dict.values():
    for title in titles_list:
        base_job_titles.add(normalize_title(title))  # Add the normalized base title



# Function to find the best matching title from the base job titles
def match_title_to_base(scraped_title, base_job_titles):
    # Normalize the scraped job title
    normalized_title = normalize_title(scraped_title)
    # Check if the normalized title exactly matches one of the base job titles
    if normalized_title in base_job_titles:
        return normalized_title  # Return the matching base title
    
    # Partial match checking - longer base titles are checked first to match more specific job titles
    sorted_base_titles = sorted(base_job_titles, key=len, reverse=True)
    for base_title in sorted_base_titles:
        if base_title in normalized_title:
            if "senior" in normalized_title and "senior" not in base_title:
                return "senior "+base_title
            if "junior" in normalized_title and "junior" not in base_title:
                return "junior "+base_title
            if "chief" in normalized_title and "chief" not in base_title:
                return "chief "+base_title
            return base_title
    
    return "unmatched"  # Return "unmatched" or some default value if no match is found

# Define a function to match job titles using fuzzy string matching
def fuzzy_match_title(scraped_title, base_job_titles, threshold=90):
    # Use the process function to find the closest match above a certain score threshold
    best_match, score = process.extractOne(normalize_title(scraped_title), base_job_titles)
    # Only accept the match if the score is above the threshold
    return best_match if score >= threshold else "unmatched"

'''
# Match each scraped title to a base title
matched_titles = {title: match_title_to_base(title, base_job_titles) for title in scraped_titles}
matched_titles_fuzzy = {}
for title in [normalize_title(each) for each in scraped_titles]:
    matched_titles_fuzzy[title] = fuzzy_match_title(title, base_job_titles)

def fill_best_value(matched_titles_fuzzy,matched_titles):
    for k,v in matched_titles_fuzzy:
        if v==None:
            matched_titles_fuzzy[k]= matched_titles[k]

    for k,v in matched_titles_fuzzy:
        if v==None:
            matched_titles_fuzzy[k]= matched_titles[k]
'''


df_j_cleaned["title_cleaned"] = df_j_cleaned["title"].apply(lambda x: fuzzy_match_title(x,base_job_titles))
df_j_cleaned["title_cleaned"] = df_j_cleaned["title_cleaned"].apply(lambda x: match_title_to_base(x,base_job_titles))
df_j_cleaned["title_cleaned"] = df_j_cleaned.apply(
    lambda x: x["title_cleaned"] if x["title_cleaned"] != "unmatched" else fuzzy_match_title(x["normTitle"], base_job_titles),
    axis=1
)
df_j_cleaned["title_cleaned"] = df_j_cleaned.apply(
    lambda x: x["title_cleaned"] if x["title_cleaned"] != "unmatched" else match_title_to_base(x["normTitle"], base_job_titles),
    axis=1
)
#df_j_cleaned["displayTitle"] = df_j_cleaned["displayTitle"].apply(lambda x: fuzzy_match_title(x,base_job_titles))
#df_j_cleaned["displayTitle"] = df_j_cleaned["displayTitle"].apply(lambda x: match_title_to_base(x,base_job_titles))

#df_j_cleaned["title"] = df_j_cleaned["title"].apply(lambda x: match_title_to_base(x,base_job_titles))

columns_most_important = ['company', 'truncatedCompany',
 'companyRating',
 'companyReviewCount',
  'searched_title',
    'title',
 'title_cleaned',
 'normTitle',
 'min',
 'max',
 'type',
 'snippet',
 'jobDescription',
 'skills_tagged']

df_j_cleaned = df_j_cleaned[columns_most_important]

#df_j_cleaned.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_cleaned_v2.csv")


In [80]:
df_j_cleaned =  pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_cleaned_v2.csv")


In [76]:
#pd.set_option('display.max_rows', 500)


condition =df_j_cleaned["title_cleaned"]=="unmatched"
df_j_cleaned.loc[condition,"title_cleaned"]=df_j_cleaned.loc[condition,"searched_title"]
df_j_cleaned["title_cleaned"] = df_j_cleaned["title_cleaned"].apply(lambda x:x.lower()) 

df_j_cleaned["title_cleaned"] = df_j_cleaned.apply(
    lambda x: x["title_cleaned"] if x["title_cleaned"] != "unmatched" else fuzzy_match_title(x["searched_title"], base_job_titles),
    axis=1
)

df_j_cleaned["title_cleaned"]=df_j_cleaned["title_cleaned"].apply(lambda x:x.replace("(cto)","").strip())
df_j_cleaned["title_cleaned"]=df_j_cleaned["title_cleaned"].apply(lambda x:x.replace("(cio)","").strip())
df_j_cleaned["title_cleaned"].unique()


array(['chief intelligence officer', 'chief information officer',
       'chief strategy officer', 'chief technology officer',
       'chief product officer', 'senior data architect', 'data architect',
       'senior ai developer', 'enterprise architect',
       'big data architect', 'data engineer', 'senior big data engineer',
       'database administrator', 'ai architect',
       'senior machine learning engineer', 'machine learning engineer',
       'data scientist', 'ai project manager',
       'senior research scientist', 'research scientist',
       'principal data scientist', 'research analyst',
       'quantitative research manager', 'principal scientist',
       'business intelligence analyst', 'business intelligence manager',
       'business analyst', 'senior of business intelligence',
       'business analysis manager',
       'senior business intelligence analyst',
       'senior data product manager', 'product analyst', 'data analyst',
       'director of product managem

In [81]:
from sklearn.preprocessing import StandardScaler
def fill_na_type(min_sal,max_sal,type):
    if pd.isna(type):
        if min_sal//1000 or max_sal//1000:
            return "yearly"
        else:
            return "hourly"
    else:
        return type
df_j_cleaned["type"]=df_j_cleaned.apply(lambda x: fill_na_type(x["min"],x["max"],x["type"]),axis=1)


def adjust_max_salary(min_sal,type):
    if type=="yearly":
        return min_sal+10000
    else:
        return min_sal+10
    
df_j_cleaned["max"]=df_j_cleaned[["min","type"]].apply(lambda x :adjust_max_salary(x["min"],x["type"]),axis=1)

for each in df_j_cleaned["title_cleaned"].unique():
    condition = df_j_cleaned["title_cleaned"]==each
    # Condition for filtering ColumnB

    # Calculate the average of ColumnB based on the condition
    average_value_min = df_j_cleaned.loc[condition, 'min'].mean()
    average_value_max = df_j_cleaned.loc[condition, 'max'].mean()

    # Fill missing values in ColumnA with this average
    df_j_cleaned.loc[condition,"min"] = df_j_cleaned.loc[condition,"min"].fillna(average_value_min)
    df_j_cleaned.loc[condition,"max"] = df_j_cleaned.loc[condition,"max"].fillna(average_value_max)

# Assuming a standard work year of 40 hours/week and 52 weeks/year
HOURS_PER_WEEK = 40
WEEKS_PER_YEAR = 52
condition = df_j_cleaned["type"]=="hourly"
df_j_cleaned.loc[condition,"min"]=df_j_cleaned.loc[condition,"min"]*HOURS_PER_WEEK * WEEKS_PER_YEAR
df_j_cleaned.loc[condition,"max"]=df_j_cleaned.loc[condition,"max"]*HOURS_PER_WEEK * WEEKS_PER_YEAR
df_j_cleaned.loc[condition,"type"]="yearly"
df_j_cleaned["min"].fillna(df_j_cleaned["min"].mean(),inplace=True)
df_j_cleaned["max"].fillna(df_j_cleaned["max"].mean(),inplace=True)



# Assuming df is your DataFrame and 'current_salary' is the salary column
scaler = StandardScaler()
df_j_cleaned['normalized_min'] = scaler.fit_transform(df_j_cleaned[['min']])
df_j_cleaned['normalized_max'] = scaler.fit_transform(df_j_cleaned[['max']])


In [82]:

#df_j_cleaned.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_cleaned_all_titles_salary.csv")
#df_j_cleaned.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_cleaned_salary.csv")


In [34]:
## clean course data 


df_c_cleaned = df_c
df_c_cleaned.columns.tolist()



def salary_to_dict(row):
    if pd.isna(row):
        # Return a dictionary with NaN values if the row is NaN
        return {'max': np.nan, 'min': np.nan, 'type': np.nan}
    elif isinstance(row, dict):
        # Return the row as is if it's already a dictionary
        return row
    else:
        # If the row is a string representation of a dictionary (assumed if not NaN or dict),
        # safely evaluate it to a dictionary here
        # Note: Be cautious with `eval`. Here it's mentioned for potential string to dict conversion.
        # In a secure context, confirm the string format and consider `ast.literal_eval` instead.
        try:
            # Convert string to dictionary safely
            dict_row = eval(row)
            return dict_row if isinstance(dict_row, dict) else {'max': np.nan, 'min': np.nan, 'type': np.nan}
        except:
            # In case of error during eval, return NaN values
            return {'max': np.nan, 'min': np.nan, 'type': np.nan}

def  course_price_detail(x):
    if pd.isna(x):
        return np.nan
    else:
        cou_dict = ast.literal_eval(x)
        return cou_dict["amount"]

def  category_primary(x):
    cou_dict = ast.literal_eval(x)
    return cou_dict["url"]

def  labels_return_title(x):
    cou_dict = ast.literal_eval(x)
    if len(cou_dict)==0:
        return "UnDefined"
    return cou_dict[0]["title"]



def  labels_return_url(x):
    cou_dict = ast.literal_eval(x)
    if len(cou_dict)==0:
        return "UnDefined"
    return cou_dict[0]["url"]


def  labels_return_display_name(x):
    cou_dict = ast.literal_eval(x)
    if len(cou_dict)==0:
        return "UnDefined"
    return cou_dict[0]["display_name"]


## Lemmitization and cleaning columns

df_c_cleaned["price"] = df_c_cleaned["price_detail"].apply(lambda x: course_price_detail(x))
df_c_cleaned["requirements_data"] = df_c_cleaned["requirements_data"].apply(lambda x:ast.literal_eval(x)[0])
df_c_cleaned["course_title"]=df_c_cleaned["labels"].apply(lambda x:labels_return_title(x))
df_c_cleaned["course_url"] =df_c_cleaned["labels"].apply(lambda x:labels_return_url(x))
df_c_cleaned["course_display_name"] =df_c_cleaned["labels"].apply(lambda x:labels_return_display_name(x))
df_c_cleaned["objectives"] =df_c_cleaned["objectives"].apply(lambda x:ast.literal_eval(x)[0])
del df_c_cleaned["labels"]
df_c_cleaned["what_you_will_learn_data"] = df_c_cleaned["what_you_will_learn_data"].apply(lambda x:ast.literal_eval(x)[0])
df_c_cleaned["target_audiences"] = df_c_cleaned["target_audiences"].apply(lambda x:ast.literal_eval(x)[0])
del df_c_cleaned["price_detail"]

## merge all description into one    

#clean course data 
def clean_course_df(df):
    df = df.reset_index(drop=True)
    # Create an empty DataFrame with a column named 'single_space' that has one row with a space
    df_space = pd.DataFrame({"single_space": [" "] * len(df)})
    
    df["full_description"] = df["title"]+df_space["single_space"]+df["description"]+df_space["single_space"]+df["headline"]+df_space["single_space"]+df["what_you_will_learn_data"]+df_space["single_space"]+df["objectives"]
    df = clean_lem_stop(df,"full_description")
    course_description = df["full_description"].tolist()
    return df,course_description

clean_course_columns =["title",
"description",
"headline",
"requirements_data",
"what_you_will_learn_data",
"target_audiences",
"objectives",
#"full_description",
"course_title",
"course_display_name"]
for clean_column in clean_course_columns:
    df_c_cleaned[clean_column]=df_c_cleaned[clean_column].astype(str)
    df_c_cleaned = clean_lem_stop(df_c_cleaned,clean_column)

df_c_cleaned,course_descriptions = clean_course_df(df_c_cleaned)



df_c_cleaned_2 = df_c_cleaned.copy()
df_c_cleaned_2["skills_tagged"] = df_c_cleaned_2["full_description"].apply(lambda x:phrase_matcher_model(x,skills_list))





In [36]:
course_columns_imp = [
 'title',
 'url',
 'description',
 'headline',
 'num_subscribers',
 'rating',
 'num_reviews',
 'num_quizzes',
 'num_lectures',
 'num_curriculum_items',
 'requirements_data',
 'what_you_will_learn_data',
 'target_audiences',
 'estimated_content_length',
 'content_info',
 'instructional_level',
 'objectives',
 'full_description',
 'skills_tagged']

df_c_cleaned_2 = df_c_cleaned_2[course_columns_imp]

In [37]:
## save processed datasets 

df_c_cl_backup =  df_c_cleaned_2.copy()
#df_j_cleaned.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_cleaned.csv")
#df_c_cleaned_2.to_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/course_cleaned.csv")


#df_c_cleaned = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/course_cleaned.csv")
#df_j_cleaned=pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_cleaned.csv")

#import ast
#df_j_cleaned["skills_tagged"]=df_j_cleaned["skills_tagged"].apply(lambda x:ast.literal_eval(x))

In [60]:
df_j_cleaned

,company,truncatedCompany,companyRating,companyReviewCount,searched_title,title,title_cleaned,normTitle,min,max,type,snippet,jobDescription,skills_tagged
0,Community Hospital,Community Hospital,3.1,39.0,Chief Intelligence Officer,system chief quality officer regulatory compli...,unmatched,Chief Quality Officer,NaN,NaN,NaN,This position is responsible for facilitating ...,updated pay scales august looking experienced ...,{}
1,Mesalands Community College,Mesalands Community College,3.8,14.0,Chief Intelligence Officer,chief financial officer,unmatched,Chief Financial Officer,65000.00,90000.00,yearly,FULL-TIME/ PART-TIME: Full Time.\n Mesalands C...,mesalands community college position announcem...,{applied science}
2,Cornerstone Healthcare Group,Cornerstone Healthcare Group,3.3,5192.0,Chief Intelligence Officer,chief operating officer chief clinical officer,unmatched,Chief Operating Officer,NaN,NaN,NaN,Our team plays a vital role in the recovery pr...,scionhealth opportunity chief operating office...,{}
3,Clinical Management Consultants,Clinical Management Consultants,4.2,13.0,Chief Intelligence Officer,hospital chief financial officer,unmatched,Chief Financial Officer,250897.00,276569.00,yearly,The Hospital Vice President Finance will be re...,beloved hospital organization south texas acti...,{c}
4,Clinical Management Consultants,Clinical Management Consultants,4.2,13.0,Chief Intelligence Officer,nurse chief quality officer,unmatched,Chief Quality Officer,115.00,115.00,hourly,The Nurse Chief Quality Officer will be respon...,outstanding medical center searching strong nu...,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,Bloomberg,Bloomberg,3.9,952.0,Chief Technology Officer (CTO),product organizational coach cto coo office,unmatched,Chief Operating Officer,145000.00,210000.00,yearly,Coach product managers and team members on ear...,bloomberg cto office future looking technical ...,{data centers}
7615,Backblaze,Backblaze,3.5,2.0,Chief Technology Officer (CTO),portfolio manager pmo office cto contract,unmatched,Portfolio Manager,105000.00,150000.00,yearly,Backblaze makes storing and using data astonis...,backblaze backblaze makes storing data astonis...,"{cloud operations, problem solving}"
7616,Overhead Door Corporation,Overhead Door Corporation,3.2,386.0,Chief Technology Officer (CTO),chief technology officer,chief technology officer,Chief Technology Officer,80968.68,97510.66,yearly,Lead Innovation Engineering team that will res...,position position reports genie company presid...,{cloud infrastructure}
7617,Bloomberg,Bloomberg,3.9,952.0,Chief Technology Officer (CTO),applied researcher search cto office,chief technology officer,Chief Technology Officer,240000.00,330000.00,yearly,Partner across the organization to develop a l...,bloomberg cto office future looking technical ...,"{deep learning, cloud computing, natural langu..."


In [ ]:
df_c.shape

(3353, 34)